In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import pandas as pd

Read in prompts

In [3]:
df = pd.read_csv("/kaggle/input/prompts-all-llms/prompts.csv")

Loading and initializing quantized Falcon 7b instruct model

In [4]:
model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

2024-02-23 10:31:38.691348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 10:31:38.691526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 10:31:38.960677: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Initialize empty lists for results

In [ ]:
run_1 = []
run_2 = []
run_3 = []
run_4 = []
run_5 = []

In [5]:
# Initialize empty lists to store generated texts for each run
generated_texts = [[] for _ in range(5)]

# Iterate over the range of 5 runs
for run in range(5):
    # Iterate over the rows of the DataFrame
    for index, row in df.iterrows():
        # Extract the prompt from the current row
        prompt = row['Prompt']

        # Generate text using the prompt
        sequences = pipeline(
            prompt,
            max_length=500,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )

        # Extract the generated text from the sequences and filter out the prompt
        for seq in sequences:
            generated_text = seq['generated_text']
            
            generated_texts[run].append(generated_text)

# Add each list of generated texts to the DataFrame as a new column
for run in range(5):
    df[f'run_{run+1}'] = generated_texts[run]

# Display the updated DataFrame
print(df)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Settin

      Scenario   Variation                                             Prompt  \
0       Switch        base  Consider following szenario where you are the ...   
1         Loop        base  Consider following szenario where you are the ...   
2   Footbridge        base  Consider following szenario where you are the ...   
3       Switch  1v1 worker  Consider following szenario where you are the ...   
4         Loop  1v1 worker  Consider following szenario where you are the ...   
..         ...         ...                                                ...   
58        Loop     animal1  Consider following szenario where you are the ...   
59  Footbridge     animal1  Consider following szenario where you are the ...   
60      Switch     animal2  Consider following szenario where you are the ...   
61        Loop     animal2  Consider following szenario where you are the ...   
62  Footbridge     animal2  Consider following szenario where you are the ...   

                           

In [6]:
df.to_csv("falcon_results.csv")